# CHAPTER 03. 평가
성능 평가 지표(Evaluation Metric) : 일반적으로 모델이 회귀냐에 따라 여러 종류로 나뉜다. 
- 회귀의 경우 대부분 실제값과 예측값의 오차 평균값에 기반한다. 기본적으로 예측 오차를 가지고 정규화 수준을 재 가공하는 방법하는 방법이 회귀의 성능 평가 지표 유형이다..
- 분류 성능 평가 지표
    - 정확도(Accuracy)
    - 오차행렬(Confusion Matrix)
    - 정밀도(Precision)
    - 재현율(Recall)
    - F1 스코어
    - ROC AUC
- 분류는 결정 클래스 값 종류이 유형에 따라 2개의 결괏값만을 가지는 이진 분류와 여러개의 결정 클래스 값을 가지는 멀티분류로 나뉠 수 있다.

## 01. 정확도(Accuracy)
: 실제 데이터에서 예측에티어가 얼마나 같은지를 판단하는 지표
- 예측 결과가 동일한 데이터 건수 / 전체 예측 데이터 건수
- 직관적으로 모델 예측 성능을 나타내는 평가 지표이다. 하지만 이진분류의 경우 데이터 구성에 따라 ML 모델 성능을 왜곡할 수 있기 때문에 정확도 수치 하나만 가지고 성능을 평가하기 어렵다.

- BaseEstimator을 상속받으면 Customized 형태의 Estimator를 개발자가 생성할 수 있다.

In [2]:
# 정확도 지표가 어떻게 ML 모델 성능을 왜곡하는지 알아보기
# BaseEstimator 클래스를 상속받아 아무런 학습을 하지 않고, 성별에 따라 생존자를 예측하는 단순한 Classifier를 생성
from sklearn.base import BaseEstimator

class MyDummyClassifier(BaseEstimator):
    # fit()메서드는 아무 것도 학습하지 않는다.
    def fit(self, X, y=None):
        pass
    # predict()메서드는 단순히 Sex 피처가 1이면 0, 그렇지 않으면 1로 예측함
    def predict(self, X):
        pred = np.zeros((X.shape[0], 1))
        for i in range(X.shape[0]) :
            if X['Sex'].iloc[i] == 1:
                pred[i] = 0
            else :
                pred[i] = 1
                
        return pred

In [ ]:
# MyDummyClassifier를 이용해 타이타닉 생존자 예측 수행
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 원본 데이터를 재로딩, 데이터 가공, 학습 데이터/테스트 데이터 분할.
titanic_df = pd.read_csv('/Users/wizdom/Desktop/data_analysis/Kaggle_titanic/titanic/train.csv')
y_titanic_df = titanic_df['Survived']
X_titanic_df = titanic_df.drop('Survived', axis = 1)
X_titanic_df = transform_features(X_titanic_df)
X_train, X_test, y_train, y_test = train_test_split(X_titanic_df, y_titanic_df, test_size = 0.2, random_state=0)

# 위에서 생성한 Dummy Classifier를 이용해 학습/예측/평가 수행
myclf = MyDummyClassifier()
myclf.fit(X_train, y_train)

mypredictions = myclf.predict(X_test)
print('Dummy Classifier의 정확도는: {0:.4f}'.format(accuracy_score(y_test, mypredictions)))

Dummy Classifier의 정확도 : 0.7877

이렇게 단순한 알고리즘으로 예측을 하더라도 데이터의 구성에 따라 정확도의 결과는 78%로 높은 수치가 나올 수 있기에 정확도를 평가지표로 사용할 때는 매우 신중해야한다. 특히 정확도는 불균한 레이블 값 분포에서 ML 모델의 성능을 판단할 경우, 적합한 평가 지표가 아니다.

In [12]:
# load_digits() API를 통해 MINIST 데이터 세트를 변환해 불균형한 데이터 세트로 만든 뒤에 정확도 지표 적용시 어떤 문제가 발생할 수 있는지 살펴보자
# 레이블 값이 0부터 9까지 있는 멀티 레이블 분류가 되어있는 것을 레이블 값이 7인 것만 True, 나머지는 모두 Flase로 변환해 이진 분류문제로 바꾼다.
# 즉, 전체 데이터의 10% TRUE, 나머지 90%는 False인 불균형한 데이터 세트로 변형
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

class MyFakeClassifier(BaseEstimator):
    def fit(self, X, y):
        pass
    
    # 입력값으로 들어오는 X 데이터 세트의 크기만큼 모두 0값으로 만들어서 반환
    def predict(self, X):
        return np.zeros((len(X), 1), dtype=bool)

# 사이킷런의 내장 데이터 세트인 load_digits()를 이용해 MNIST 데이터 로딩
digits = load_digits()

# digits 번호가 7번이면 True이고 이를 astype(int)로 1로 변환, 7번이 아니면 False이고 0으로 변환.
y = (digits.target == 7).astype(int)
X_train, X_test, y_train, y_test = train_test_split(digits.data, y, random_state=11)

In [14]:
# 불균형한 레이블 데이터 분포도 확인.
print('레이블 테스트 세트 크기: ', y_test.shape)
print('테스트 세트 레이블 0과 1의 분포도:\n',pd.Series(y_test).value_counts())

# Dummy Classfier로 학습/예측/정확도 평가
fakeclf = MyFakeClassifier()
fakeclf.fit(X_train, y_train)
fakepred = fakeclf.predict(X_test)
print('모든 예측을 0으로 하여도 정확도는: {:.3f}'.format(accuracy_score(y_test, fakepred)))

레이블 테스트 세트 크기:  (450,)
테스트 세트 레이블 0과 1의 분포도:
 0    405
1     45
dtype: int64
모든 예측을 0으로 하여도 정확도는: 0.900


이처럼 정확도 평가 지표는 불균형한 레이블 데이터 세트에서는 성능 수치로 사용돼서는 안된다.
- 이러한 한계점을 극복하기 위해 여러가지 분류 지표와 함께 사용해야한다.

## 02. 오차 행렬
: 학습된 분류 모델이 예측을 수행하면서 얼마나 헷갈리고 있는지도 함께 보여준다. 즉, 이진 분류의 예측 오류가 얼마인지와 더불어 어떠한 유형의 예측 오류가 발생하고 있는지를 함께 나타내는 지표이다. 
- 이진 분류에서 성능지표로 잘 활용된다.
- 4분면 행렬에서 실제 레이블 클래스 값과 예측 레이블 클래스 값이 어떠한 유형을 가지고 매핑되는지를 나타낸다.
    - TN, FP, FN, TP는 예측 클래스와 실제 클래스의 Positive 결정값(값 1)과 Negative 결정 값(값 0)의 결합에 따라 결정된다.
    - 4분면의 왼쪽, 오른쪽을 예측된 클래스 값 기준으로 Negative와 Positive로 분류
    - 4분면의 위, 아래를 실제 클래스값 기준으로 Negative와 Positive로 분류
    - 그럼 예측 클래스와 실제 클래스 값 유혀엥 따라 결정되는 TN, FP, FN, TP 형태로 오차 행렬의 4분면을 채운다.
    - 4가지를 다양하게 결합해 분류 모델 예측 성능의 오류가 어떠한 모습으로 발생하는지 알 수 있다. 
- TN, FP, FN, TP
    - TN : 예측 값을 Negative 값 0으로 예측했고, 실제 값 역시 Negative 값 0
    - FP : 예측 값을 Positive 값 1로 예측했는데, 실제 값은 Negative 값 0
    - FN : 예측 값을 Negative 값 0으로 예측했는데, 실제 값은 Positive 값 1
    - TP : 예측 값을 Positive 값 1로 예측했는데, 실제 값 역시 Positive 값 1

- 오차행렬을 구하기 위한 API : confusion_matrix()

In [15]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, fakepred)

array([[405,   0],
       [ 45,   0]])

In [16]:
# 출력된 오차 행렬은 ndarray 형태
# TN : [0, 0]로 405, FP: [0, 1]로 0, FN : [1, 0]로 45, TP: [1, 1]로 0

- TP, TN, FP, TN 값은 Classifier 성능의 여러 면모를 판단할 수 있는 기반 정보를 제공한다.
- 이 값들을 조합해 분류 성능을 측정할 수 있는 주요 지표인 정확도, 정밀도, 재현율 값을 알 수 있다.
- 정확도 = 예측 결과와 실제 값이 동일한 건수 / 전체 데이터 수  = (TN + TP)/(TN + FP + FN + TP)
    - 오차 행렬에서 True에 해당하는 값인 TN과 TP에 좌우된다.

- 일반적으로 불균형한 레이블 클래스를 가지는 이진 분류 모델에서는 많은 데이터 중에서 중점적으로 찾아야 하는 매우 적은 수의 결괏값에 Positive를 설정해 1값을 부여하고, 그렇지 않은 경우는 Negative로 0 값을 부여하는 경우가 많다.
    - 불균형한 이진 분류 데이터 세트에서는 Positive 데이터 건수가 매우 작기 때문에 데이터에 기반한 ML 알고리즘 Positive보다는 Negative로 예측 정확도가 높아지는 경향이 발생(9,990건이 Negative이고 100건이 Positive라면 N으로 예측하는 경향이 더 강해서 TN은 매우 커지고 TP는 매우 작아지게 된다.
    - N으로 예측할 때 정확도가 높기 때문에 FN이 매우 작고, P로 예측하는 경우가 작기 때문에 FP 역시 매우 작아진다.
    - 결과적으로 정확도 지표는 비대칭한 데이터 세트에서 P에 대한 예측 정확도를 판단하지 못한 채 N에 대한 예측 정확도만으로도 분류의 정확도가 매우 높게 나타나는 수치적인 판단
  

- 정확도 분류 모델의 성능을 측정할 수 있는 한 가지 요소일 뿐이다.

## 03. 정밀도와 재현율
: Positive 데이터 세트의 예측 성능에 좀 더 초점을 맞춘 평가 지표이다.
- 정밀도 : "예측"을 Positive로 한 대상 중에 에측과 실제 값이 Positive로 일치한 데이터의 비율.
    - TP / (FP + TP)
    - 공식의 분모인 FP + TP는 예측을 Positive로 한 모든 데이터 건수이며, 공식의 분자인 TP는 예측 값과 실제 값이 Positive로 일치한 데이터 건수이다.
    - Positive 예측 성능을 더욱 정밀하게 측정하기 위한 평가 지표로 양성 예측도라고도 불린다.
- 재현율 : "실제 값"이 Positive인 대상 중에 예측과 실제 값이 Positive로 일치한 데이터의 비율
    - TP / (FP + TP)
    - 공식의 분모인 FN + TP는 실제 값이 Positive인 모든 데이터 건수이며, 공식의 분자인 TP는 예측과 실제 값이 Positive로 일치한 데이터 건수이다.
    - 민감도(Sensitive) 또는 TPR(True positive Rate)라고 불린다.
    
    
- 정밀도와 재현율 지표 중에 이진 분류 모델의 업무 특성에 따라서 특정 평가 지표가 더 중요한 지표로 간주될 수 있다. 
    - 재현율이 상대적으로 더 중요한 지표인 경우는, 실제 Positive 양성인 데이터 예측을 Negative로 잘못 판단하게 되면 업무상 큰 영향이 발생하는 경우이다.
        - 예) 실제 P인 암환자를 P 양성이 아닌 N 음성으로 잘못 판단 했을 경우, 실제 금융거래 사기인 P 건을 N로 잘못 판단한 경우
    - 정밀도가 상대적으로 더 중요한 지표인 경우는, 실제 Negative 음성인 데이터 예측을 Positive 양성으로 잘못 판단하게 되면 업무상 큰 영향이 발생하는 경우이다.
    - 재현율과 정밀도 모두 TP를 높이는데 동일하게 초점을 맞추지만,
        - 재현율은 FN(실제 P, 예측 N)를 낮추는 데,
        - 정밀도는 FP(실제 N, 예측 P)를 낮추는 데 초점을 맞춘다.
        
        
- 이같은 특성떄문에 재현율과 정밀도는 서로 보완적인 지표로 분류의 성능을 평가하는데 적용된다.
    - 가장 좋은 성능 평가는 재현율과 정밀도 모두 높은 수치를 얻는 것이다.
    - 반면에 둘 주 ㅇ어느 한 평가 지표만 매우 높고, 다른 수치는 매우 낮은 결과를 나타내는 경우는 바람직하지 않다.